In [1]:
# echo-server.py
# import selectors
# from _thread import *
# import threading
import socket
import time
from random import randint
import json 
import uuid
import os

'''
Program variables
'''
LAN_IP = "192.168.1.255"  # Standard loopback interface address (localhost)
PORT = 9994  # Port to listen on (non-privileged ports are > 1023)
EVENT_TIMEOUT = 10 #s #time before an emergency event times out regardless of acks
ACK_TIMEOUT = 4 #s time before emergency message is broadcasted to un-acked vehicles
PROXIMITY_PURGE_TIME = 5 #s
PROXIMITY_THRESHOLD = 10 # 1 dimensional coordinate, meters

#unused function. May be used for threading/user io
def thread(Client, Pserver):
    while True:
        # data received from client
        data = Pserver.recv(1024)
        if data:
            Client.send(data)
        elif not data:
            print_lock.release()
            break

'''
Vehicle class
Does the buld of the heavy lifting for
variable storage, send/recieve routines, variable updates and maintanance
Takes in a defined IP and port number if none-default values are desired
-Defaults to loopback ip/port 9999
-Modify this with vm subnet broadcast IP/port, and optional test variables
'''
class Vehicle:
    def __init__(self, LAN_IP="127.0.0.1", PORT=9999, velocity=1, location=0):
        '''
        initializes things specific to this instance, with defaults set above.
        Initializes networking ports with UDP/broadcast and prints errors.
        Note tx will error out if you try to assign a none broadcast ip address on the LAN
        '''
        self.location = location # 1 dimensional x coordinate
        self.last_update = time.time()
        self.msg = ""
        self.buffer = ""
        self.proximity_table = {}
        self.velocity = velocity # m/s
        self.LAN_IP = LAN_IP
        self.PORT = PORT
        self.id = str(uuid.uuid1())
        #self.broadcast = 
        try:
            self.rx = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
            self.tx = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
            self.tx.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1) #set broadcast
            self.rx.bind((LAN_IP, PORT))
            self.rx.settimeout(.3) #set timeout on socket to 1s. Val to be lowered for better latency
        except Exception as e:
            print(e) 
    
    #Unused functionality created for threading vehicle instances
    def create_transmit_socket(self):
        init_socket = None
        try:
            init_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
            init_socket.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1) #set broadcast
        except Exception as e:
            print(e) 
        return init_socket
    
    #acts like a vector addition
    def update_location(self):
        current_time = time.time()
        self.location = self.location + self.velocity * (current_time - self.last_update)
        self.last_update = current_time
        return self.location

    
    def broadcast_location(self):
        try:
            self.update_location()
            broadcast_loc_msg = {"msg_type": "location_update", "time": time.time(), "vehicle_id": self.id, 
            "location": str(self.location)}
            self.tx.sendto(json.dumps(broadcast_loc_msg).encode(), (LAN_IP, PORT))
        except Exception as e:
            print(e) 
        return "location sent"
        

    def send_event_msg(self, msg):
        try:
            self.tx.sendto(msg, (LAN_IP, PORT))
        except Exception as e:
            print(e)
        return "sent event msg"
    
    
    def recieve(self):
        try:
            self.recieve_data = self.rx.recv(1024)
            if self.recieve_data:
                return self.recieve_data
            elif not self.recieve_data:
                return ''
        except Exception as e:
            #print(e)
            return ''
    
    '''
    Update the table if we're closer than PROXIMITY_THRESHOLD
    Chose a dictionary because it allows us to easily enter new keys/keep track of values for entry times
    '''
    def update_proximity_table(self, parsed_dic):
        self.update_location()
        proximity = abs(float(parsed_dic["location"]) - float(self.location))
        vehicle_id = parsed_dic["vehicle_id"]
        
        if vehicle_id != self.id and proximity < PROXIMITY_THRESHOLD:
            self.proximity_table[vehicle_id] = time.time()
            print("updating  proximity table")
            print(self.proximity_table)
    
    
    def clean_proximity_table(self):
        #Forces a memcopy of dictionary keys so we don't step on an array we're iterating over
        for key in list(self.proximity_table.keys()):
            current_time = time.time()
            if (current_time - self.proximity_table[key]) > PROXIMITY_PURGE_TIME:
                print("Deleting table key instance")
                try:
                    del self.proximity_table[key]
                except Exception as e:
                    print(e)
    
    
    def parse(self, data_dict):
        if data_dict["msg_type"] == "location_update":
            self.update_proximity_table(data_dict)
        elif data_dict["msg_type"] == "crash":
            if self.id in data_dict["proximity_table"]:
                self.send_ack()
    
    
    def send_ack(self):
        ack = json.dumps({"msg_type": "ack", "vehicle_id": self.id}).encode()
        self.tx.sendto(ack, (LAN_IP, PORT))


#------------------------------------------------------------------------------------- end of Vehicle class



'''
Emergency message definition
Creates an emergency event message and broadcasts message
The definition then monitors for acks coming in and resends messages as needed
The 2 main loops are for sending/resending the event message, and monitoring for replies. 
The inner loop also looks for other event messages and acknoledges is it is in the message. This is important to prevent event timeouts
- accepts vehicle instance, returns result string
'''
def emergency_event(vehicle):
    # initial variables for vehicles we want acks from 
    # and timeout value
    ack_table = vehicle.proximity_table.copy() #careful... =dictionaries is treated as a pointer
    current_time = time.time()
    
    #Keep sending emergency requests until we have all of the acks we want or we hit a timeout
    while True:
        emergency_msg = {"msg_type": "crash", "time": current_time, "vehicle_id": vehicle.id, 
            "proximity_table": ack_table}
        print("sending emergency msg, ack table:")
        print(ack_table)
        vehicle.send_event_msg(json.dumps(emergency_msg).encode())
        ack_start = time.time()
        #Listen for messages and remove ack ids from table as they come in
        if ack_table:
            while (time.time() - ack_start) < ACK_TIMEOUT:
                #check for message timeout and quite if it's exceeeded
                
                data = vehicle.recieve()
                if data:
                    data_dic = json.loads(data.decode())
                    ack_id = data_dic["vehicle_id"]
                    if data_dic["msg_type"] == "ack" and ack_id != vehicle.id:
                        print("recieved data in event fxn")
                        if ack_id in ack_table: #delete expected ack if the ack id is in the table
                            try:
                                del ack_table[ack_id]
                                print("ack recieved and removed from proximity table, new table:")
                                print(ack_table)
                            except Exception as e:
                                print("failed to remove id from event proximity table")
                                print(e)
                    elif data_dic["msg_type"] == "crash" and ack_id != vehicle.id:
                        print("sending ack for another crash")
                        vehicle.parse(data_dic)
                else:
                    print("no data recieved: " + str(time.time()))
                if not ack_table:
                    print("emergency event ack success")
                    return "recieved acks for all vehicles"
            print("ack timed out")
            if (time.time() - current_time) > EVENT_TIMEOUT:
                    print("Failed to recieve acks from vehicles")
                    return "Failed to recieve acks from vehicles"
        else:
            print("emergency event success")
            return "recieved acks for all vehicles"




In [2]:

'''
This is the __main__ program... of you wanted to impliment it in that way.

Establishes vehicle instance with parameters specified at the top of the program.
p_loops is % of the broadcast loops that generates an emergency event. Currently 20% for testing
Consider modifying this for multiple VM runs so you don't birthday attack the probability

'''
vehicle = Vehicle(LAN_IP=LAN_IP, PORT=PORT, velocity=1.3, location = 0)
i = 100
p_loops = 20 #percent of loops
while True:
    #Broadcast hey this is me and where I am
    vehicle.broadcast_location()
    print("location: " + str(vehicle.location))
    for n in range(6):
        #read from buffer (x - my owm broadcast msg) times
        vehicle.clean_proximity_table()
        data = vehicle.recieve()
        if data:
            #if we recieve data, load it into a dictionary and parse to see if
            #    we need to update our proximity table or ack an emergency message
            #Note that the dictionary lets you easily add/discard extra functionality
            # e.g. multihops, responses etc... harder in c...
            data_dict = json.loads(data.decode())
            msg_type = vehicle.parse(data_dict)
    rand = randint(0, i)
    print("rand int: " + str(rand))
    if rand < p_loops: #should run every p_loops percentage of loops
        print("emergency event triggered")
        emergency_event(vehicle)
    time.sleep(2)

location: 0.007600450515747071
updating  proximity table
{'c463f57c-76aa-11ed-810e-b91ea026c07f': 1670471133.045319}
rand int: 4
emergency event triggered
sending emergency msg, ack table:
{'c463f57c-76aa-11ed-810e-b91ea026c07f': 1670471133.045319}
recieved data in event fxn
ack recieved and removed from proximity table, new table:
{}
emergency event ack success
location: 4.685513162612915
updating  proximity table
{'c463f57c-76aa-11ed-810e-b91ea026c07f': 1670471136.0141008}
rand int: 73
location: 8.950419092178345
updating  proximity table
{'c463f57c-76aa-11ed-810e-b91ea026c07f': 1670471139.0708728}
rand int: 44
location: 13.45373468399048
updating  proximity table
{'c463f57c-76aa-11ed-810e-b91ea026c07f': 1670471142.2521758}
rand int: 84
location: 17.74132821559906
updating  proximity table
{'c463f57c-76aa-11ed-810e-b91ea026c07f': 1670471145.246351}
rand int: 78
location: 21.77211287021637
updating  proximity table
{'c463f57c-76aa-11ed-810e-b91ea026c07f': 1670471148.4045062}
updating 

RuntimeError: dictionary changed size during iteration